In [ ]:
import neurokit2 as nk
import pandas as pd
import os
import re
import warnings
import statistics

In [ ]:
def replace_channel(cols_template, channel):
    '''
    Function to modify a template of column names to adapt to each signal.

    Input:
    cols_template (list of str): A list of column names containing a placeholder "[channel]".
    channel (str): The name of the channel to replace the placeholder.

    Returns:
    list of str: A list of column names with the "[channel]" placeholder replaced by the provided channel name.
    '''
    
    return [re.sub(r"\[channel\]", channel, col) for col in cols_template]

In [ ]:
def process_stdevs_s2(file_path, bunch_of_cols, filename):
    '''
    Function to process standard deviations of 5-minutes baselines of SBP and DBP from a CSV file.

    Input:
    file_path (str): The path to the CSV file containing the SBP and DBP data.
    bunch_of_cols (list of lists of str): A list of lists where each sublist contains column names for SBP and DBP data.
    filename (str): The name of the file to be used as the index for the resulting DataFrame.

    Returns:
    pd.DataFrame: A DataFrame with calculated standard deviations of for each minute from 5-minute baselines of SBP and DBP. 
    The index is based on the provided filename, and the columns are defined by the input `bunch_of_cols`.
    '''
    i = 0
    bn_col = 0
    df_sub = pd.read_csv(file_path)
    marker = 201 #Entire baseline is marked as "201"
                               
    dataframes = []
    sbp_dbp_cols = [col for sublist in bunch_of_cols for col in sublist]
    sbp_df = pd.DataFrame(columns=sbp_dbp_cols)
    sbp_df.loc[0] = [None] * len(sbp_dbp_cols)
    
    if bn_col == 0: #start with SBP
        curr_idx = df_sub.index[df_sub['marker'] == marker] #determining the first index for the given marker in df
            
            
                
    for y in range(0, 300000, 60000): #calculations initiated from a 5-minute baseline, with minute intervals
        sbp = statistics.stdev(df_sub["SBP"].iloc[curr_idx[0]+y:curr_idx[0]+y+60000]) 
        sbp_df.iloc[0, i] = sbp #saving calculations in local df
        i += 1 #moving to the next column
        
    bn_col += 1
    
    if bn_col == 1: #proceed to DBP
       
        curr_idx = df_sub.index[df_sub['marker'] == marker]
        for y in range(0, 300000, 60000):
            dbp = statistics.stdev(df_sub["DBP"].iloc[curr_idx[0]+y:curr_idx[0]+y+60000]) 
            sbp_df.iloc[0, i] = dbp #saving calculations in local df
            i += 1

    
    sbp_df.index = [filename.replace('.csv', '')] #saving to df with the subject's id as the index name
    return sbp_df

In [ ]:
cols_sbp_db = ['ECG_Rate_mean', 'HRV_RMSSD', 'SBP', 'DBP', 'CO', 'TPR', 'wr', 'tl', 'tr']
list_of_channels = ['SBP', 'DBP', 'marker']
cols_template = ["baseline_visit2_min1_[channel]_stdev", "baseline_visit2_min2_[channel]_stdev", "baseline_visit2_min3_[channel]_stdev", "baseline_visit2_min4_[channel]_stdev", "baseline_visit2_min5_[channel]_stdev"]
sbp_cols = replace_channel(cols_template, list_of_channels[0])
dbp_cols = replace_channel(cols_template, list_of_channels[1])
bunch = [sbp_cols, dbp_cols]

In [ ]:
# Processing all CSV files in the folder
input_folder = '/home/ubuntu/eSportData/jupyter-data/VU_AMS/s2_output/'
output_df = pd.DataFrame()

for file_name in os.listdir(input_folder):  #iterating through the list with the databases of the subjects
    if file_name.endswith('.csv'):
        try:
            print(file_name)
            file_path = os.path.join(input_folder, file_name)
            processed_df = process_stdevs_s2(file_path, bunch, file_name) #calling the function to create df with calculated standard deviations of SBP and DBP baselines
            output_df = pd.concat([output_df, processed_df]) #adding the subject's data from the iteration to the collective df with other subjects
        except:
            pass
# Saving the resulting DataFrame to a CSV file
output_df.to_csv('/home/ubuntu/eSportData/jupyter-data/VU_AMS/processed_stdevs_s2.csv')